In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,8170.50,-0.13,-0.03,-0.02,0.00,-0.00,0.0,-0.00
3,-0.13,1199.11,0.01,-0.01,0.00,-0.00,-0.0,0.00
4,-0.03,0.01,498.56,-0.00,0.01,0.00,-0.0,-0.00
5,-0.02,-0.01,-0.00,131.83,-0.00,-0.00,0.0,-0.00
6,0.00,0.00,0.01,-0.00,30.28,0.00,0.0,0.00
7,-0.00,-0.00,0.00,-0.00,0.00,6.84,-0.0,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,1.5,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.0,0.27


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00002,0.00000,-0.00001,0.00001,-0.00003
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00005,0.00005,-0.00001,-0.00005
5,-0.00002,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00002,-0.00000
6,0.00000,0.00001,0.00005,-0.00001,1.00000,0.00002,0.00000,0.00006
7,-0.00001,-0.00001,0.00005,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00001,-0.00002,-0.00001,0.00002,0.00000,-0.00001,1.00000,0.00003
9,-0.00003,0.00000,-0.00005,-0.00000,0.00006,-0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18799943374055064

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.34912285e+06 1.77063238e+05 2.95163519e+04 1.98654587e+03
 9.84775618e+01 4.49072718e+00 1.63041171e-01 1.38180748e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999730,-0.021416,-0.008719,-0.002297,-0.000527,-0.000119,-0.000026,-0.000005
1,0.020573,0.996176,-0.082922,-0.017787,-0.004029,-0.000911,-0.000200,-0.000035
2,0.010281,0.081504,0.994832,-0.058316,-0.012307,-0.002767,-0.000607,-0.000109
3,0.003215,0.022103,0.055760,0.996230,-0.061172,-0.012959,-0.002822,-0.000504
4,0.000915,0.006232,0.015015,0.059021,0.994811,-0.079563,-0.016268,-0.002887
5,0.000273,0.001859,0.004463,0.016934,0.076188,0.990197,-0.114177,-0.018964
6,0.000088,0.000599,0.001437,0.005437,0.023709,0.107387,0.977701,-0.178816
7,0.000032,0.000219,0.000527,0.001991,0.008666,0.038369,0.175476,0.983695


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0002701479761653136,
 0.003823699166465633,
 0.005167791410024658,
 0.0037701950412235563,
 0.005189342277725939,
 0.009803459815241333,
 0.022299331265146827,
 0.016304696354222936]